<a href="https://colab.research.google.com/github/kamusone/BITCOIN-PR-DICTION-UTILIZATION-SENTIMENT-ANALYSE/blob/master/Bitcoin_Spark_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
# UNCOMMENT THIS IF YOU'RE USING GOOGLE COLAB!

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install numpy
!pip install -q pandas==0.24.1
!pip install -q seaborn==0.9
!pip install -q pyspark==2.3.3
!pip install -q findspark
!pip install -q handyspark

# AFTER RUNNING THIS CELL, YOU MUST RESTART THE RUNTIME TO USE UPDATED VERSIONS OF PACKAGES!

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import os
import numpy as np
import findspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
# from handyspark import *

findspark.init()
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[*] pyspark-shell'


spark = SparkSession.builder.getOrCreate()

sql = SQLContext(spark)

In [5]:
!pip install tweet-preprocessor
!pip install vaderSentiment
!pip install textblob

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

def sentiment_analysis(text):
    """Determine if sentiment is positive, negative, or neutral
    algorithm to figure out if sentiment is positive, negative or neutral
    uses sentiment polarity from TextBlob, VADER Sentiment and
    sentiment from text-processing URL
    could be made better :)
    """
    text =str(text)
    # pass text into TextBlob
    text_tb = TextBlob(text)

    # pass text into VADER Sentiment
    analyzer = SentimentIntensityAnalyzer()
    text_vs = analyzer.polarity_scores(text)


    if text_tb.sentiment.polarity <= 0 and text_vs['compound'] <= -0.5:
        sentiment = "negative"  # very negative
    elif text_tb.sentiment.polarity <= 0 and text_vs['compound'] <= -0.1:
        sentiment = "negative"  # somewhat negative
    elif text_tb.sentiment.polarity == 0 and text_vs['compound'] > -0.1 and text_vs['compound'] < 0.1:
        sentiment = "neutral"
    elif text_tb.sentiment.polarity >= 0 and text_vs['compound'] >= 0.1:
        sentiment = "positive"  # somewhat positive
    elif text_tb.sentiment.polarity > 0 and text_vs['compound'] >= 0.1:
        sentiment = "positive"  # very positive
    else:
        sentiment = "neutral"

    # calculate average polarity from TextBlob and VADER
    polarity = (text_tb.sentiment.polarity + text_vs['compound']) / 2

    return sentiment
  
def polarity_analysis(text):
    """Determine if sentiment is positive, negative, or neutral
    algorithm to figure out if sentiment is positive, negative or neutral
    uses sentiment polarity from TextBlob, VADER Sentiment and
    sentiment from text-processing URL
    could be made better :)
    """
    text =str(text)
    # pass text into TextBlob
    text_tb = TextBlob(text)

    # pass text into VADER Sentiment
    analyzer = SentimentIntensityAnalyzer()
    text_vs = analyzer.polarity_scores(text)


    if text_tb.sentiment.polarity <= 0 and text_vs['compound'] <= -0.5:
        sentiment = "negative"  # very negative
    elif text_tb.sentiment.polarity <= 0 and text_vs['compound'] <= -0.1:
        sentiment = "negative"  # somewhat negative
    elif text_tb.sentiment.polarity == 0 and text_vs['compound'] > -0.1 and text_vs['compound'] < 0.1:
        sentiment = "neutral"
    elif text_tb.sentiment.polarity >= 0 and text_vs['compound'] >= 0.1:
        sentiment = "positive"  # somewhat positive
    elif text_tb.sentiment.polarity > 0 and text_vs['compound'] >= 0.1:
        sentiment = "positive"  # very positive
    else:
        sentiment = "neutral"

    # calculate average polarity from TextBlob and VADER
    polarity = (text_tb.sentiment.polarity + text_vs['compound']) / 2

    return polarity
  
def sentiment_scores(text):
  text=str(text)
  analyzer = SentimentIntensityAnalyzer()
  text_vs = analyzer.polarity_scores(text)
  return ([text_vs['neg'], text_vs['neu'], text_vs['pos'], text_vs['compound']])

In [6]:
df = pd.read_csv('/content/drive/My Drive/sentiment_analysis/tweets.csv', sep = ',')
# df.text = df.text.astype('str')
df = df.drop_duplicates(subset='text', keep='first')
df=df.drop(columns=['user', 'fullname', 'tweet-id', 'url', 'likes', 'replies',
       'retweets', 'html'])
df = df.dropna()
df.head()

,timestamp,text
0,2016-06-25 23:59:52,Bitcoin: Pissed-off customers sue GAW Miners i...
1,2016-06-25 23:59:36,I'm not gone lie I be staring at pplwhat btc's...
2,2016-06-25 23:59:17,Did you miss Beyond Bitcoin Hangout? I CANNOT ...
3,2016-06-25 23:59:15,What are the merits and risks of investing in ...
4,2016-06-25 23:59:08,Hash Ocean #Bitcoin Cloud Mining №1 in the wor...


In [0]:
df.to_csv('/content/drive/My Drive/sentiment_analysis/tweets_drop.csv', sep = ',', index = False)

In [10]:
df1 = pd.read_csv('/content/drive/My Drive/sentiment_analysis/tweets_drop.csv', sep = ',',  lineterminator='\n')
df1.head()

,timestamp,text
0,2016-06-25 23:59:52,Bitcoin: Pissed-off customers sue GAW Miners i...
1,2016-06-25 23:59:36,I'm not gone lie I be staring at pplwhat btc's...
2,2016-06-25 23:59:17,Did you miss Beyond Bitcoin Hangout? I CANNOT ...
3,2016-06-25 23:59:15,What are the merits and risks of investing in ...
4,2016-06-25 23:59:08,Hash Ocean #Bitcoin Cloud Mining №1 in the wor...


In [0]:
tweets=sql.createDataFrame(df1)

In [12]:
!pip install langdetect
from langdetect import detect

def langdetection(cleaned):
    try :
        if detect(cleaned) == 'en':
            return cleaned
        else :
            return np.nan
    except:
        return np.nan

     |████████████████████████████████| 1.0MB 3.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
import preprocessor as p
import re
def clean_tweets(input_str):
    input_str = re.sub(r'RT', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())

In [14]:
func_udf = udf(clean_tweets, StringType())
tweets = tweets.withColumn('cleaned', func_udf(tweets['text']))
tweets.show(3)

+-------------------+--------------------+--------------------+
|          timestamp|                text|             cleaned|
+-------------------+--------------------+--------------------+
|2016-06-25 23:59:52|Bitcoin: Pissed-o...|Bitcoin Pissed of...|
|2016-06-25 23:59:36|I'm not gone lie ...|I m not gone lie ...|
|2016-06-25 23:59:17|Did you miss Beyo...|Did you miss Beyo...|
+-------------------+--------------------+--------------------+
only showing top 3 rows



In [15]:
func_udf = udf(langdetection, StringType())
tweets= tweets.withColumn('lang_detect', func_udf(tweets['cleaned']))

func_udf = udf(polarity_analysis, FloatType())
tweets = tweets.withColumn('polarity', func_udf(tweets['cleaned']))

func_udf = udf(sentiment_analysis, StringType())
tweets = tweets.withColumn('sentiment', func_udf(tweets['cleaned']))

tweets.show(3)

+-------------------+--------------------+--------------------+--------------------+--------+---------+
|          timestamp|                text|             cleaned|         lang_detect|polarity|sentiment|
+-------------------+--------------------+--------------------+--------------------+--------+---------+
|2016-06-25 23:59:52|Bitcoin: Pissed-o...|Bitcoin Pissed of...|Bitcoin Pissed of...|-0.26845|  neutral|
|2016-06-25 23:59:36|I'm not gone lie ...|I m not gone lie ...|I m not gone lie ...|     0.0|  neutral|
|2016-06-25 23:59:17|Did you miss Beyo...|Did you miss Beyo...|Did you miss Beyo...|-0.04855|  neutral|
+-------------------+--------------------+--------------------+--------------------+--------+---------+
only showing top 3 rows



In [0]:
func_udf = udf(sentiment_scores, ArrayType(FloatType()))
tweets = tweets.withColumn('p_neg', func_udf(tweets['cleaned'])[0])
tweets = tweets.withColumn('p_neu', func_udf(tweets['cleaned'])[1])
tweets = tweets.withColumn('p_pos', func_udf(tweets['cleaned'])[2])
tweets = tweets.withColumn('p_comp', func_udf(tweets['cleaned'])[3])

In [0]:
tweets = tweets.selectExpr("timestamp","cleaned","lang_detect","sentiment","polarity","p_neg","p_neu","p_pos","p_comp")

In [0]:
tweets_final = tweets.dropna(subset=['lang_detect'])

In [0]:
tweets_final = tweets_final.withColumn("date",tweets_final['timestamp'].cast(TimestampType()))
# tweets_final = tweets_final.selectExpr("date","cleaned","lang_detect","sentiment","polarity","p_neg","p_neu","p_pos","p_comp")
tweets_final = tweets_final.selectExpr("date","p_neg","p_neu","p_pos","p_comp")

In [20]:
tweets_final.write.csv('/content/drive/My Drive/sentiment_analysis/tweets_final.csv')

KeyboardInterrupt: ignored

In [0]:
tweets_final.repartition(1).write.format('com.databricks.spark.csv').save("/content/drive/My Drive/sentiment_analysis/tweets_final1.csv",header = 'true')

AttributeError: ignored